# Attempt: Transfer Learning with InceptionResNet and Data Augmentation

In [ ]:
import pandas as pd
import numpy as np
from time import time
import datetime
import tensorflow as tf
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import backend as K

%matplotlib inline
print(tf.keras.__version__)
print(K.tensorflow_backend._get_available_gpus())

SEED = 42
SESSION_NAME = datetime.datetime.now().strftime("%Y%m%d-%H%M")
PI = tf.constant(np.pi)


In [ ]:
traindf=pd.read_csv("./labels_all_added_shifted.csv",dtype={'id': str, 'label_rad': np.float32, 'label_scaled': np.float32, 'label_scaled_shifted': np.float32, 'label_sin': np.float32, 'label_cos': np.float32})
traindf.head()

In [ ]:
#Visualizing some Examples
IMG_PATH = "images/"
fig = plt.figure(figsize=(10,5))
for i in range (1, 5):
    a = fig.add_subplot(2, 2, i)
    imgplot = plt.imshow(mpimg.imread(IMG_PATH+traindf.iloc[(i*5000)-1]['id']), cmap='gray', vmin=0, vmax=1)
    a.set_title(traindf.iloc[(i*5000)-1]['label_rad']) 
    plt.tight_layout()

In [ ]:
# Input constants
HEIGHT = 480
WIDTH = 640
TARGET_HEIGHT = 240
TARGET_WIDTH = 320
CHANNELS = 3
BATCH_SIZE = 8
VAL_SPLIT = 0.01
NUM_IMAGES = 55479
NUM_TRAIN_IMAGES = NUM_IMAGES*(1-VAL_SPLIT)
NUM_VAL_IMAGES = NUM_IMAGES*VAL_SPLIT

datagen = keras.preprocessing.image.ImageDataGenerator(validation_split=VAL_SPLIT,
                                                       rescale = 1/256,
                                                       width_shift_range = 10,
                                                       height_shift_range = 10,
                                                       rotation_range = 5,
                                                       brightness_range = [0.5, 1.5],
                                                       zoom_range = 0.2)

train_generator = datagen.flow_from_dataframe(dataframe=traindf,
                                                    directory=IMG_PATH,
                                                    x_col="id",
                                                    y_col="label_scaled",
                                                    subset="training",
                                                    seed=SEED,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    class_mode="other",
                                                    target_size=(TARGET_HEIGHT,TARGET_WIDTH),
                                                    color_mode = "rgb")

val_generator = datagen.flow_from_dataframe(dataframe=traindf,
                                                    directory=IMG_PATH,
                                                    x_col="id",
                                                    y_col="label_scaled", #evt. None
                                                    subset="validation",
                                                    seed=SEED,
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    class_mode="other", #evt. None
                                                    target_size=(TARGET_HEIGHT,TARGET_WIDTH),
                                                    color_mode = "rgb")

fig = plt.figure(figsize=(12,10))
for X_batch, y_batch in train_generator:
    for i in range(1, 9):
        image = ((X_batch[i-1]*256).astype('uint8'))#[:,:,0]
        a = fig.add_subplot(4, 2, i)
        imgplot = plt.imshow(image, cmap='gray', vmin = 0, vmax = 256)
        a.set_title("{}".format(i))
        plt.tight_layout()
    break

In [ ]:
#Objective Function
def scaled_and_shifted_squared_angdiff(y_true, y_pred):
    y_true_scaled = (y_true+0.5)*2*PI
    y_pred_scaled = (y_pred+0.5)*2*PI
    delta = tf.atan2(tf.sin(y_pred_scaled-y_true_scaled), tf.cos(y_pred_scaled-y_true_scaled))
    loss = tf.square(delta)
    return loss

In [ ]:
#Test the objective Function
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
feed_dict={x: [-0.49, 0], y: [0.49, 0.5]}
with tf.Session() as sess:
    print(sess.run(scaled_and_shifted_squared_angdiff(x, y), feed_dict))

In [ ]:
#Model Definition
model = Sequential()
model.add(InceptionResNetV2(include_top=False,
                            weights='imagenet',
                            input_shape=(TARGET_HEIGHT,TARGET_WIDTH,3),
                            pooling='avg'))
model.add(Dense(128, activation="elu"))
model.add(Dropout(0.05, seed=SEED))
model.add(Dense(64, activation="elu"))
model.add(Dense(1, activation="linear"))

model.layers[0].trainable = False
model.summary()

In [ ]:
# Training constants
NUM_EPOCHS = 2
LEARNING_RATE = 0.0003
DECAY = 1e-6

FILEPATH_SAVE='checkpoints/{}_model_weights.h5'.format(SESSION_NAME)
checkpoint = keras.callbacks.ModelCheckpoint(FILEPATH_SAVE,
                                             monitor=[scaled_and_shifted_squared_angdiff],
                                             verbose=1,
                                             mode='max')

#Tensorboard Callback
FILEPATH_TENSORBOARD_LOG = 'logdir/{}'.format(SESSION_NAME)
tensorboard = keras.callbacks.TensorBoard(log_dir = FILEPATH_TENSORBOARD_LOG,
                                          histogram_freq=1,
                                          batch_size=BATCH_SIZE,
                                          write_graph=True,
                                          write_grads=True,
                                          write_images=True,
                                          update_freq=NUM_TRAIN_IMAGES//5
                                         )

model.compile(optimizer='adam', #keras.optimizers.rmsprop(lr=LEARNING_RATE, decay=DECAY),
                loss=scaled_and_shifted_squared_angdiff,
                metrics=[scaled_and_shifted_squared_angdiff])

#starting Tensorboard:
#Navigated the browser to http://localhost:8088
#cd to current directory
#%tensorboard --logdir=logdir/ --host localhost --port 8088

In [ ]:
history = model.fit_generator(train_generator, 
                               epochs=NUM_EPOCHS,
                               workers=4, 
                               steps_per_epoch=NUM_TRAIN_IMAGES // BATCH_SIZE, 
                               validation_data = val_generator,
                               validation_steps = NUM_VAL_IMAGES // BATCH_SIZE,
                               shuffle=True,
                               callbacks=[tensorboard, checkpoint])

In [ ]:
pred=model.predict_generator(val_generator, verbose=1)
filenames=val_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":pred.tolist()})
results.to_csv("results_{}.csv".format(SESSION_NAME),index=False)
results.head()